![alt text](./Cerny_logo_1.jpg)

# Analysis of Cerny ventilation recordings

The data processed and analysed in this Notebook were collected by the **Neonatal Emergency and Transport Service of the Peter Cerny Foundation**, Budapest, Hungary

**Author: Dr Gusztav Belteki**


## Analysis of mechanically ventilated cases

Explorative data analysis of **145 ventilated cases** among recordings `AL000001 - AL000300`. 

- It calculates statistics on clinical details of ventilated cases and exports them as Excel files and as graphs. 
- It identifies ventilator modes, recordings with multiple ventilation modes and in those, the dominant ventilator mode; exports Excel files and graphs of these. 
- It calculates descriptive statistics on various ventilator parameters in the individual recordings and writes them to Excel files in different format (grouping).
- It produces time series graphs on various ventilator parameters and exports them.

Imported: 

**data_pars_measurements_ventilated_1_300.pickle,  data_pars_settings_ventilated_1_300.pickle, data_pars_alarms_ventilated_1_300.pickle, vent_modes_ventilated_1_300.pickle, clin_df_pickle_1_300.pickle**, **Fabian_parameters.xlsx**

Exported: 

- Excel files and graphs about clinical data and ventilator modes are exported to **/Users/guszti/ventilation_fabian/Analyses/analysis_ventilated_1_300** folder
- Time series graphs on ventilator parameters are exported to the **/Volumes/GUSZTI/data_dump/fabian/fabian_cases** folder into individual subfolders named after the recording
- **vent_modes_ventilated_1_300_plus.pickle** (additional data about multiple ventilator modes and dominant modes in in the DataFrame

### Importing the necessary libraries and setting options

In [ ]:
import IPython
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import seaborn as sns
import sklearn as sk

import os
import sys
import re
import pickle

from scipy import stats
from pandas import Series, DataFrame
from datetime import datetime, timedelta

%matplotlib inline
matplotlib.style.use('classic')
matplotlib.rcParams['figure.facecolor'] = 'w'

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
# pd.set_option('mode.chained_assignment', None) 

In [ ]:
print("Python version: {}".format(sys.version))
print("pandas version: {}".format(pd.__version__))
print("matplotlib version: {}".format(matplotlib.__version__))
print("NumPy version: {}".format(np.__version__))
print("SciPy version: {}".format(sp.__version__))
print("IPython version: {}".format(IPython.__version__))
print("scikit-learn version: {}".format(sk.__version__))

### List and set the working directory and the directory to write out data

In [ ]:
# Topic of the Notebook which will also be the name of the subfolder containing results
TOPIC = 'fabian'

# Name of the external hard drive
DRIVE = 'GUSZTI'

# Directory containing clinical and blood gas data
CWD = '/Users/guszti/ventilation_fabian'

# Directory on external drive to read the ventilation data from
DIR_READ = '/Volumes/%s/Fabian/fabian_data' % DRIVE

DIR_WRITE = '%s/%s/%s' % (CWD, 'Analyses', 'analysis_ventilated_1_300')
if not os.path.isdir(DIR_WRITE):
    os.makedirs(DIR_WRITE)

# Images and raw data will be written on an external hard drive
if not os.path.isdir('/Volumes/%s/data_dump/%s' % (DRIVE, TOPIC)):
    os.makedirs('/Volumes/%s/data_dump/%s' % (DRIVE, TOPIC))
DATA_DUMP = '/Volumes/%s/data_dump/%s' % (DRIVE, TOPIC)

In [ ]:
os.chdir(CWD)
os.getcwd()

In [ ]:
DIR_READ, DIR_WRITE, DATA_DUMP

### Import ventilator and clinical data from pickle archives

In [ ]:
# Import ventilator parameters, settings and alarms

with open('%s/%s.pickle' % (DATA_DUMP, 'data_pars_measurements_ventilated_1_300'), 'rb') as handle:
    data_pars_measurements_ventilated = pickle.load(handle)
    
with open('%s/%s.pickle' % (DATA_DUMP, 'data_pars_settings_ventilated_1_300'), 'rb') as handle:
    data_pars_settings_ventilated = pickle.load(handle)
    
with open('%s/%s.pickle' % (DATA_DUMP, 'data_pars_alarms_ventilated_1_300'), 'rb') as handle:
    data_pars_alarms_ventilated = pickle.load(handle)


# Import DataFrame with ventilation modes

with open('%s/%s.pickle' % (DATA_DUMP, 'vent_modes_ventilated_1_300'), 'rb') as handle:
    vent_modes_ventilated = pickle.load(handle)


# Import clinical data

with open('%s/%s.pickle' % (DATA_DUMP, 'clin_df_1_300'), 'rb') as handle:
    clin_df = pickle.load(handle)

In [ ]:
len(data_pars_measurements_ventilated)

In [ ]:
cases = sorted(data_pars_measurements_ventilated.keys())

### Import table for interpreting ventilator parameters

In [ ]:
par_key_table = pd.read_excel('Fabian_parameters.xlsx')
par_key_table;

## Statistics on clinical details of these cases

In [ ]:
len(clin_df)

In [ ]:
clin_df_ventilation = clin_df.loc[vent_modes_ventilated.index]

In [ ]:
clin_df_ventilation.head(2)

In [ ]:
clin_df_ventilation.info()

In [ ]:
clin_df_ventilation_stats = round(clin_df_ventilation.describe(), 2)
clin_df_ventilation_stats

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'clinical_data_ventilated_1_300.xlsx'))
clin_df_ventilation.to_excel(writer, 'ventilated_cases')
clin_df_ventilation_stats.to_excel(writer, 'stats')
writer.save()

In [ ]:
fig, ax = plt.subplots(figsize = (6, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['gestational age', 'corrected gestational age']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([clin_df_ventilation['Gestational Age (weeks)'] ,
             clin_df_ventilation['Corrected gestational Age (weeks)']],
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_ylabel('weeks', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
ax.set_xticklabels(xticklabels)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'ventilated_gest_age_1_300', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1,);

In [ ]:
fig, ax = plt.subplots(figsize = (6, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['birth weight', 'actual weight']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([clin_df_ventilation['Birth Weight'], clin_df_ventilation['Weight']],
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_ylim(0, 5500)
ax.set_ylabel('grams', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'ventilated_weight_1_300', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1,);

In [ ]:
fig, ax = plt.subplots(figsize = (3, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['recording duration']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot(clin_df_ventilation['Duration'] / (60 * 1E+9), 
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_ylabel('minutes', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'ventilation_duration_1_300', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1,);

## How many cases with the different ventilation modes

### Which recordings had more than one ventilation modes

In [ ]:
multiple_mode = [case for case in cases if
        (vent_modes_ventilated.loc[case][['IPPV', 'PSV', 'SIMV', 'SIMVPSV', 'SIPPV']] != 0).sum() > 1]

In [ ]:
len(multiple_mode)

In [ ]:
vent_modes_ventilated['multiple_mode'] = np.where(vent_modes_ventilated.index.isin(multiple_mode), 'Yes', 'No')

In [ ]:
vent_modes_ventilated.head()

In [ ]:
vent_modes_ventilated.loc[multiple_mode]

In [ ]:
def autolabel(rects):
    """
    Attach a text label above each bar displaying its height
    """
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2., 1.05*height,
                '%d' % int(height), ha='center', va='bottom', size = 14)

### Considering all modes used during recordings

In [ ]:
print('SIMV:', sum(vent_modes_ventilated['SIMV'] > 0))
print('SIPPV:', sum(vent_modes_ventilated['SIPPV'] > 0))
print('SIMVPSV:', sum(vent_modes_ventilated['SIMVPSV'] > 0))
print('PSV:', sum(vent_modes_ventilated['PSV'] > 0))
print('IPPV:', sum(vent_modes_ventilated['IPPV'] > 0))
print('VG_on:', sum(vent_modes_ventilated['VG_on'] > 0))
print('total', len(vent_modes_ventilated))

In [ ]:
dpi = 300
filetype = 'jpg'
labels = ['SIMV', 'SIPPV', 'SIMV-PSV', 'IPPV', 'PSV']
xticks = np.arange(len(labels))
width = 0.6

fig, ax = plt.subplots(figsize = [6,4])
rects = plt.bar(xticks, [sum(vent_modes_ventilated['SIMV'] > 0), sum(vent_modes_ventilated['SIPPV'] > 0), 
                         sum(vent_modes_ventilated['SIMVPSV'] > 0), sum(vent_modes_ventilated['IPPV'] > 0),
                         sum(vent_modes_ventilated['PSV'] > 0)], 
                        width=width, color='black', alpha  = 0.75, align = 'center')

ax.set_xlabel('ventilation mode', size = 14)
ax.set_xticks(xticks)
ax.set_xticklabels(labels, size = 14, rotation = 0)
ax.set_ylabel('number of cases', size = 14)
ax.set_title('Ventilation mode used')
ax.set_ylim(0, 130)
ax.grid(True)

autolabel(rects)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'vent_modes_ventilated_1_300', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1,);

### Considering only the predominant ventilation modes

In [ ]:
vent_modes_ventilated['dominant_mode'] = \
    vent_modes_ventilated[['IPPV', 'PSV', 'SIMV', 'SIMVPSV', 'SIPPV']].idxmax(axis = 1)

In [ ]:
vent_modes_ventilated.head()

In [ ]:
vent_modes_ventilated['dominant_mode'].value_counts()

In [ ]:
vent_modes_ventilated['dominant_mode'].value_counts().index

In [ ]:
dpi = 300
filetype = 'jpg'
xticklabels = vent_modes_ventilated['dominant_mode'].value_counts().index
xticks = np.arange(len(vent_modes_ventilated['dominant_mode'].value_counts()))
fig, ax = plt.subplots(figsize = [4,4])

rects = plt.bar(xticks, vent_modes_ventilated['dominant_mode'].value_counts(), color='black',
                                            width = 0.7, alpha  = 0.75, )
ax.set_xticks(xticks)
ax.set_xticklabels(xticklabels, size = 14, rotation = 0)
ax.set_xlabel('ventilation mode', size = 14)
ax.set_ylabel('number of cases', size = 14)
ax.set_title('Predominant ventilation mode')
ax.set_ylim(0, 130)
ax.grid(True)

autolabel(rects)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'vent_modes_ventilated_dominant_mode_1_300', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1,);

### Considering only the recordings with a single ventilator mode

In [ ]:
sum(vent_modes_ventilated['multiple_mode'] == 'No')

In [ ]:
only_mode = vent_modes_ventilated[vent_modes_ventilated['multiple_mode'] == 'No']['dominant_mode'].value_counts()
only_mode

In [ ]:
dpi = 300
filetype = 'jpg'
xticklabels = only_mode.index
xticks = np.arange(len(only_mode))
fig, ax = plt.subplots(figsize = [4,4])

rects = plt.bar(xticks, only_mode, color='black', width = 0.7, alpha  = 0.75)
ax.set_xticks(xticks)
ax.set_xticklabels(xticklabels, size = 14, rotation = 0)
ax.set_xlabel('ventilation mode', size = 14)
ax.set_ylabel('number of cases', size = 14)
ax.set_title('Cases with a single ventilator mode')
ax.set_ylim(0, 110)
ax.grid(True)

autolabel(rects)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'vent_modes_ventilated_only_mode_1_300', filetype), dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1,);

### How many recordings had VG ventilation and for how long

In [ ]:
# How many nan values
sum(vent_modes_ventilated['VG_on'].isnull())

In [ ]:
# has some VG ventilation
sum(vent_modes_ventilated['VG_on'] > 0)

In [ ]:
# has no VG ventilation
len(vent_modes_ventilated) - sum(vent_modes_ventilated['VG_on'] > 0)

In [ ]:
has_VG = vent_modes_ventilated[vent_modes_ventilated['VG_on'] > 0]

In [ ]:
# VG was on throughout the whole recording 
only_VG = has_VG[has_VG['VG_on'] == has_VG['total']]
len(only_VG)

In [ ]:
only_VG['dominant_mode'].value_counts()

#### Save extended file about ventilation modes to pickle archive

In [ ]:
with open('%s/%s.pickle' % (DATA_DUMP, 'vent_modes_ventilated_1_300_plus'), 'wb') as handle:
    pickle.dump(vent_modes_ventilated, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Check the format of the ventilator data

In [ ]:
data_pars_measurements_ventilated['AL000008'].info()

In [ ]:
data_pars_settings_ventilated['AL000008'].info()

Some settings are in non-numeric format. Convert them to float

In [ ]:
par_list = ['PIP_set', 'PEEP_set', 'FiO2_set', 'Flow_insp_set', 'Flow_exp_set', 'Ti_set', 'Te_set', 'RR_set',
            'IE_I_set', 'IE_E_set', 'VG_set', 'Trigger_sens_set', 'PIP_lim_high_set', 'PIP_lim_low_set',
            'FiO2_flush_time_set', 'VG_set_kg']

for recording in data_pars_settings_ventilated:
    for par in par_list:
        if par in data_pars_settings_ventilated[recording].columns:
            data_pars_settings_ventilated[recording][par] = \
                data_pars_settings_ventilated[recording][par].astype('float')

In [ ]:
data_pars_settings_ventilated['AL000008'].info()

In [ ]:
data_pars_alarms_ventilated['AL000008'].info()

### Resample the data to calculate 1-minute means

In [ ]:
data_pars_measurements_ventilated_1min_mean = {}
data_pars_settings_ventilated_1min_mean = {}

for case in cases:
    data_pars_measurements_ventilated_1min_mean[case] = \
        data_pars_measurements_ventilated[case].resample('1min').mean()
    data_pars_settings_ventilated_1min_mean[case] = \
        data_pars_settings_ventilated[case].resample('1min').mean()

# Statistics on ventilatory parameters of these cases

## Descriptive statistics on `measured ventilator parameters`

### Statistics on individual cases

In [ ]:
stats_pars_measurements_ventilated = {} 

for case in cases:
    stats_pars_measurements_ventilated[case] = \
        round(data_pars_measurements_ventilated[case].describe(percentiles = 
                                                    (0.05, 0.25, 0.5, 0.75, 0.95)), 2)
    stats_pars_measurements_ventilated[case].index = ['data_points', 'mean', 'SD', 'min', '5pc', 
                                                      '25pc', 'median', '75pc', '95pc', 'max']

In [ ]:
stats_pars_measurements_ventilated['AL000008']

In [ ]:
# Create table with statistics for all cases and all relevant parameters
stats_pars_measurements_ventilated_all = pd.concat(stats_pars_measurements_ventilated, axis = 1).T

In [ ]:
# Remove measured parameters not given in case of mechanical ventilation
stats_pars_measurements_ventilated_all.dropna(how = 'all', subset = ['mean', 'SD', 'min', '5pc', 
                                '25pc', 'median', '75pc', '95pc', 'max'], axis = 0, inplace = True)

In [ ]:
stats_pars_measurements_ventilated_all.info()

In [ ]:
stats_pars_measurements_ventilated_all.head()

In [ ]:
stats_pars_measurements_ventilated_all.loc['AL000003']

In [ ]:
stats_pars_measurements_ventilated_all.swaplevel(0,1).loc['VTemand_resp_kg'].head()

### Statistics on individual parameters

In [ ]:
# selected individual parameters
parameters = ['C20_C', 'Cdyn', 'FiO2', 'Leak', 'MAP',  'MV_kg', 'MVresp', 'PEEP', 'PIP','R', 'RR', 
              'Trigger', 'VTemand_kg', 'VTespon_pat_kg', 'VTemand_resp_kg',  'VTimand_kg']

stats_pars_measurements_ventilated_2 = {}

for parameter in parameters:
    stats_pars_measurements_ventilated_2[parameter] = \
        stats_pars_measurements_ventilated_all.swaplevel(0,1).loc[parameter].sort_values('mean', ascending = False)

In [ ]:
stats_pars_measurements_ventilated_2['VTemand_resp_kg'].head()

In [ ]:
# Unstack table to create table for all parameters with different configuration
stats_pars_measurements_ventilated_all_2 = stats_pars_measurements_ventilated_all.unstack()

In [ ]:
stats_pars_measurements_ventilated_all_2.info()

In [ ]:
stats_pars_measurements_ventilated_all_2.head()

In [ ]:
stats_pars_measurements_ventilated_all_2['mean'].head()

### Export statistics to a multisheet Excel file and pickle archive

In [ ]:
# Save statistics into Excel file
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'stats_pars_measurements_ventilated_1_300.xlsx'))
for case in cases:
    stats_pars_measurements_ventilated[case].to_excel(writer, case)
writer.save()

In [ ]:
# Save statistics into Excel file
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'stats_pars_measurements_ventilated_1_300_2.xlsx'))
for parameter in parameters:
    stats_pars_measurements_ventilated_2[parameter].to_excel(writer, parameter)
writer.save()

In [ ]:
# Save statistics into Excel file
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'stats_pars_measurements_ventilated_all_1_300.xlsx'))
stats_pars_measurements_ventilated_all.to_excel(writer, 'stats_all_1')
stats_pars_measurements_ventilated_all_2.to_excel(writer, 'stats_all_2')
writer.save()

### Group statistics

In [ ]:
# How many data points for the various parameters in each recording
stats_pars_measurements_ventilated_all_2['data_points'].head()

In [ ]:
# Some parameters are only present in case of SIMV: VTemand, VTespon, MVresp
# Some parameters are only present in case of PSV: Ti_PSV

stats_pars_measurements_ventilated_all_2['data_points'].info()

In [ ]:
stats_pars_measurements_ventilated_all_2['mean'].describe()

In [ ]:
stats_pars_measurements_ventilated_all_2['median'].describe()

In [ ]:
percentiles = [0.001, 0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99, 0.999]

# Save statistics into Excel file
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'stats_pars_measurements_ventilated_group_1_300.xlsx'))
stats_pars_measurements_ventilated_all_2['mean'].describe(percentiles = percentiles).to_excel(writer, 'mean')
stats_pars_measurements_ventilated_all_2['median'].describe(percentiles = percentiles).to_excel(writer, 'median')
writer.save()

## Descriptive statistics on `ventilator settings`

### Statistics on individual cases

In [ ]:
stats_pars_settings_ventilated = {} 
for case in cases:
    stats_pars_settings_ventilated[case] = \
        round(data_pars_settings_ventilated[case].describe(percentiles = (0.05, 0.25, 0.5, 0.75, 0.95)), 2)
    stats_pars_settings_ventilated[case].index = ['data_points', 'mean', 'SD', 'min', '5pc', 
                                     '25pc', 'median', '75pc', '95pc', 'max']

In [ ]:
stats_pars_settings_ventilated['AL000008']

In [ ]:
# Create table with statistics for all cases and all relevant parameters
stats_pars_settings_ventilated_all = pd.concat(stats_pars_settings_ventilated, axis = 1).T

In [ ]:
# Remove measured parameters not given in case of mechanical ventilation
stats_pars_settings_ventilated_all.dropna(how = 'all', subset = ['mean', 'SD', 'min', '5pc', 
                                '25pc', 'median', '75pc', '95pc', 'max'], axis = 0, inplace = True)

In [ ]:
stats_pars_settings_ventilated_all.info()

In [ ]:
stats_pars_settings_ventilated_all.head()

In [ ]:
stats_pars_settings_ventilated_all.loc['AL000008']

### Statistics on individual settings

In [ ]:
# selected ventilator settings
parameters = ['FiO2_set', 'Flow_exp_set', 'Flow_insp_set', 'IE_E_set', 'IE_I_set',
              'MV_lim_high_set', 'MV_lim_high_set_kg', 'MV_lim_low_set',
              'MV_lim_low_set_kg', 'PEEP_set', 'PIP_lim_high_set', 'PIP_lim_low_set',
              'PIP_set', 'RR_set', 'Te_set', 'Ti_set', 'Trigger_sens_set', 'VG_set', 'VG_set_kg']

stats_pars_settings_ventilated_2 = {}

for parameter in parameters:
    stats_pars_settings_ventilated_2[parameter] = \
        stats_pars_settings_ventilated_all.swaplevel(0,1).loc[parameter].sort_values('mean', ascending = False)

In [ ]:
stats_pars_settings_ventilated_2['Ti_set'].head()

In [ ]:
# Unstack table to create table for all settings with different configuration
stats_pars_settings_ventilated_all_2 = stats_pars_settings_ventilated_all.unstack()

In [ ]:
stats_pars_settings_ventilated_all_2.info()

In [ ]:
stats_pars_settings_ventilated_all_2.head(10)

In [ ]:
stats_pars_settings_ventilated_all_2['mean'].head(10)

### Export statistics to a multisheet Excel file and pickle archive

In [ ]:
# Save statistics into Excel file
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'stats_pars_settings_ventilated_1_300.xlsx'))
for case in cases:
    stats_pars_settings_ventilated[case].to_excel(writer, case)
writer.save()

In [ ]:
# Save statistics into Excel file
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'stats_pars_settings_ventilated_1_300_2.xlsx'))
for parameter in parameters:
    stats_pars_settings_ventilated_2[parameter].to_excel(writer, parameter)
writer.save()

In [ ]:
# Save statistics into Excel file
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'stats_pars_settings_ventilated_all_1_300.xlsx'))
stats_pars_settings_ventilated_all.to_excel(writer, 'stats_all_1')
stats_pars_settings_ventilated_all_2.to_excel(writer, 'stats_all_2')
writer.save()

### Group statistics

In [ ]:
# How many data points for the various parameters in each recording
stats_pars_settings_ventilated_all_2['data_points'].head()

In [ ]:
# Some parameters are only present in case of VG ventilation: VG_set
# Some parameters are only present in case of PSV: PIP_set_PSV, Term_criteria_PSV_set
# MV limits, Apnea time were not always set
# Trigger sensitivity was not always set (no flow sensor / not synchronized??)

stats_pars_settings_ventilated_all_2['data_points'].info()

In [ ]:
stats_pars_settings_ventilated_all_2['mean'].describe()

In [ ]:
stats_pars_settings_ventilated_all_2['median'].describe()

In [ ]:
percentiles = [0.001, 0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99, 0.999]

# Save statistics into Excel file
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'stats_pars_settings_ventilated_group_1_300.xlsx'))
stats_pars_settings_ventilated_all_2['mean'].describe(percentiles = percentiles).to_excel(writer, 'mean')
stats_pars_settings_ventilated_all_2['median'].describe(percentiles = percentiles).to_excel(writer, 'median')
writer.save()

### Statistics on categorical settings

In [ ]:
parameters_2 = ['Flow_sensor_state', 'Measuring_unit_pressure_set', 
                'Oxy_sensor_state', 'Patient_range', 'Powerstate',
                'Pressure_rise_control', 'Trigger_mode', 'VG_state', 
                'Ventilation_stopped', 'Ventilator_mode', 'Ventilator_range']

In [ ]:
data_pars_settings_ventilated_combined_cat = pd.concat(data_pars_settings_ventilated, sort = True)[parameters_2]

In [ ]:
data_pars_settings_ventilated_combined_cat.head()

In [ ]:
data_pars_settings_ventilated_combined_cat.describe()

- Measuring_unit_pressure_set: always `cmH2O`
- Patient_range: always neonatal
- Pressure_rise_control: always I-flow
- Trigger_mode: always Volumetrigger
- Ventilator_range: always Neonatal


In [ ]:
data_pars_settings_ventilated_combined_cat.isnull().sum()

In [ ]:
data_pars_settings_ventilated_combined_cat['Powerstate'].value_counts()

In [ ]:
data_pars_settings_ventilated_combined_cat['Ventilation_stopped'].value_counts()

In [ ]:
data_pars_settings_ventilated_combined_cat['Ventilator_mode'].value_counts()

In [ ]:
# Create alias for long name
c = data_pars_settings_ventilated_combined_cat

In [ ]:
# Save statistics into Excel file
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'stat_pars_settings_ventilated_combined_cat_1_300.xlsx'))
data_pars_settings_ventilated_combined_cat.describe().to_excel(writer, 'categorical vars')
writer.save()

## Some unusual ventilator parameters and settings

### 1. Flow sensor state not available

In [ ]:
# This only affects 5 recordings and they are all IPPV
c[c['Flow_sensor_state'].isnull()].reset_index()['level_0'].unique()

In [ ]:
vent_modes_ventilated.loc[c[c['Flow_sensor_state'].isnull()].reset_index()['level_0'].unique()]

##### Abnormal FiO2 measurements

In [ ]:
a = stats_pars_measurements_ventilated_all_2

In [ ]:
a.head()

In [ ]:
a[a['mean']['FiO2'] < 20]

In [ ]:
# Abnormal FiO2 measurements
a[a['mean']['FiO2'] < 20]['mean']['FiO2']

In [ ]:
# FiO2 settings are appropriate
stats_pars_settings_ventilated_all_2.loc[a[a['mean']['FiO2'] < 20].index]['mean']['FiO2_set']

### 2. Very low MAP

In [ ]:
a[a['mean']['MAP'] < 6]

In [ ]:
a[a['mean']['MAP'] < 6]['mean']['MAP']

In [ ]:
low_map_list = sorted(a[a['mean']['MAP'] < 6].index)
low_map_list;

In [ ]:
# VG was on in many but not all cases
vent_modes_ventilated.loc[low_map_list].sort_values('VG_on')

In [ ]:
vent_modes_ventilated.loc[low_map_list][vent_modes_ventilated.loc[low_map_list]['VG_on'] < 100]

In [ ]:
vent_modes_ventilated.loc[low_map_list][vent_modes_ventilated.loc[low_map_list]['VG_on'] < 100].index

In [ ]:
# What was the set PIP? (In case of VG this is Pmax)
a[a['mean']['MAP'] < 6]['mean']['PIP']

In [ ]:
# What was the set PIP? (In case of VG this is Pmax)
stats_pars_settings_ventilated_all_2.loc[a[a['mean']['MAP'] < 6].index]['mean']['PIP_set']

### 3. RR not available or is 0

RR is not available for **32** out of the 145 recordings

Of the remaining 113 cases RR is always zero in **47** cases

These were the recordings that have SIMV or IPPV only

##### When is RR `NaN`

In [ ]:
stats_pars_measurements_ventilated_all_2[stats_pars_measurements_ventilated_all_2['data_points']['RR'].isnull()].head()

In [ ]:
len(stats_pars_measurements_ventilated_all_2[stats_pars_measurements_ventilated_all_2
                                             ['data_points']['RR'].isnull()])

In [ ]:
RR_nan = stats_pars_measurements_ventilated_all_2[stats_pars_measurements_ventilated_all_2['data_points']['RR'].isnull()].index

In [ ]:
vent_modes_ventilated.loc[RR_nan]

##### When is RR `0`

In [ ]:
a[a['mean']['RR'] == 0].head()

In [ ]:
len(a[a['mean']['RR'] == 0])

In [ ]:
RR_zero = a[a['mean']['RR'] == 0].index

In [ ]:
# These are all SIMV or IPPV recordings

vent_modes_ventilated.loc[RR_zero]

##### What is RR in the remaining cases

In [ ]:
RR_nan

In [ ]:
RR_zero

In [ ]:
RR_not_available = sorted(list(RR_nan) + list(RR_zero))
len(RR_not_available)

In [ ]:
RR_available = sorted(set(cases) - set(RR_not_available))
len(RR_available)

In [ ]:
vent_modes_ventilated.loc[RR_available]

In [ ]:
RR_diversity = {}

for case in RR_available:
    RR_diversity[case] = data_pars_measurements_ventilated[case]['RR'].value_counts().sort_index()

In [ ]:
RR_diversity;

In [ ]:
for case in RR_available:
    print(case, sum(data_pars_measurements_ventilated[case]['RR'] > 
                    data_pars_settings_ventilated[case]['RR_set']))

In [ ]:
for case in RR_available:
    print(case, sum(data_pars_measurements_ventilated[case]['RR'] == 
                    data_pars_settings_ventilated[case]['RR_set']))

In [ ]:
for case in RR_available:
    print(case, sum(data_pars_measurements_ventilated[case]['RR'] < 
                    data_pars_settings_ventilated[case]['RR_set']))

### 4. Mean VTemand < 3 mL/kg

In [ ]:
a[a['mean']['VTemand_kg'] < 3]

In [ ]:
a[a['mean']['VTemand_kg'] == 0]['mean']['VTemand_kg']

In [ ]:
# When VTemand was 0, the mode was always IPPV

vent_modes_ventilated.loc[a[a['mean']['VTemand_kg'] == 0].index]

In [ ]:
a[(a['mean']['VTemand_kg'] < 3) & (a['mean']['VTemand_kg'] >0)]['mean']['VTemand_kg']

In [ ]:
# The mean leak was sometimes but not always high
a[(a['mean']['VTemand_kg'] < 3) & (a['mean']['VTemand_kg'] >0)]['mean']['Leak']

### 5. Mean VTimand < 3 mL/kg

In [ ]:
a[a['mean']['VTimand_kg'] < 3]

In [ ]:
a[a['mean']['VTimand_kg'] == 0]['mean']['VTimand_kg']

In [ ]:
# When VTemand was 0, the mode was always IPPV

vent_modes_ventilated.loc[a[a['mean']['VTimand_kg'] == 0].index]

In [ ]:
a[(a['mean']['VTimand_kg'] < 3) & (a['mean']['VTimand_kg'] >0)]['mean']['VTimand_kg']

### 6. VTemand_resp (this only exists for SIMV)

In [ ]:
a[a['mean']['VTemand_resp_kg'] < 3]

In [ ]:
a[a['mean']['VTemand_resp_kg'] < 3]['mean']['VTemand_resp_kg']

### 7. VTespon_pat (this only exists for SIMV)

In [ ]:
a[a['mean']['VTespon_pat_kg'] < 1]

In [ ]:
a[a['mean']['VTespon_pat'] < 3]['mean']['VTespon_pat_kg']

### 8. MV limits are incorrectly set

In [ ]:
b = stats_pars_settings_ventilated_all_2

In [ ]:
len(b['mean']['MV_lim_low_set_kg'] < 0.1)

In [ ]:
len(b[b['mean']['MV_lim_low_set_kg'] < 0.05])

In [ ]:
len(b[b['mean']['MV_lim_high_set_kg'] > 0.5])

In [ ]:
len(b[b['mean']['MV_lim_high_set_kg'] > 1])

### 9. Trigger (volume >2 mL with volumetrigger )

In [ ]:
a[a['mean']['Trigger'] > 2]

In [ ]:
a[a['mean']['Trigger'] > 2]['mean']['Trigger']

### 10. Trigger sensitivity was not always set to minimum

In [ ]:
b[b['mean']['Trigger_sens_set'] > 1]

In [ ]:
b[b['mean']['Trigger_sens_set'] > 1]['mean']['Trigger_sens_set']

## Quality control of ventilator data in terms of duplicates and missing values

In [ ]:
vent_modes_ventilated;

### SIPPV without VG

In [ ]:
vent_modes_ventilated.loc['AL000009']

Points

- VTemand_resp and VTespon_pat is not NA as it is only for SIMV
- RR is sometimes lower than the backup RR  - how can it be what does this mean?
- Only 1.4% of duplicated rows


In [ ]:
data_pars_measurements_ventilated['AL000009'].head()

In [ ]:
data_pars_settings_ventilated['AL000009'].head()

In [ ]:
sum(data_pars_measurements_ventilated['AL000009']['RR'] > data_pars_settings_ventilated['AL000009']['RR_set'])

In [ ]:
sum(data_pars_measurements_ventilated['AL000009']['RR'] == data_pars_settings_ventilated['AL000009']['RR_set'])

In [ ]:
sum(data_pars_measurements_ventilated['AL000009']['RR'] < data_pars_settings_ventilated['AL000009']['RR_set'])

In [ ]:
x = data_pars_measurements_ventilated['AL000009']

x[x.duplicated(subset = ['PIP', 'MAP', 'PEEP', 'MV_kg', 'VTimand_kg', 'VTemand_kg',], keep = False)].head()

In [ ]:
len(x[x.duplicated(subset = ['PIP', 'MAP', 'PEEP', 'MV_kg', 'VTimand_kg', 'VTemand_kg',], keep = False)]) / len(x)

### SIPPV-VG

In [ ]:
vent_modes_ventilated.loc['AL000036']

Points

- VTemand_resp and VTespon_pat is not NA as it is only for SIMV
- RR is sometimes lower than the backup RR  - how can it be what does this mean?
- 3.8% of duplicated rows


In [ ]:
data_pars_measurements_ventilated['AL000036'].head()

In [ ]:
data_pars_settings_ventilated['AL000036'].head()

In [ ]:
sum(data_pars_measurements_ventilated['AL000036']['RR'] > data_pars_settings_ventilated['AL000036']['RR_set'])

In [ ]:
sum(data_pars_measurements_ventilated['AL000036']['RR'] == data_pars_settings_ventilated['AL000036']['RR_set'])

In [ ]:
sum(data_pars_measurements_ventilated['AL000036']['RR'] < data_pars_settings_ventilated['AL000036']['RR_set'])

In [ ]:
x = data_pars_measurements_ventilated['AL000036']

x[x.duplicated(subset = ['PIP', 'MAP', 'PEEP', 'MV_kg', 'VTimand_kg', 'VTemand_kg',], keep = False)].head()

In [ ]:
len(x[x.duplicated(subset = ['PIP', 'MAP', 'PEEP', 'MV_kg', 'VTimand_kg', 'VTemand_kg',], keep = False)]) / len(x)

### SIMV without VG

In [ ]:
vent_modes_ventilated.loc['AL000007']

Points

- RR is always higher than the backup RR  - that is OK.
- 1.7% of the rows are duplicated


In [ ]:
data_pars_measurements_ventilated['AL000007'].head()

In [ ]:
data_pars_settings_ventilated['AL000007'].head()

In [ ]:
sum(data_pars_measurements_ventilated['AL000007']['RR'] > data_pars_settings_ventilated['AL000007']['RR_set'])

In [ ]:
sum(data_pars_measurements_ventilated['AL000007']['RR'] == data_pars_settings_ventilated['AL000007']['RR_set'])

In [ ]:
sum(data_pars_measurements_ventilated['AL000007']['RR'] < data_pars_settings_ventilated['AL000007']['RR_set'])

In [ ]:
x = data_pars_measurements_ventilated['AL000007']

x[x.duplicated(subset = ['PIP', 'MAP', 'PEEP', 'MV_kg', 'VTimand_kg', 'VTemand_kg',], keep = False)].head()

In [ ]:
len(x[x.duplicated(subset = ['PIP', 'MAP', 'PEEP', 'MV_kg', 'VTimand_kg', 'VTemand_kg',], keep = False)]) / len(x)

#### SIMV-VG

In [ ]:
vent_modes_ventilated.loc['AL000008']

Points

- VTemand_resp and VTespon_pat is not NA as it is only for SIMV
- There is no RR value in DataFrame
- 2.5% of rows are duplicated


In [ ]:
data_pars_measurements_ventilated['AL000008'].head()

In [ ]:
data_pars_settings_ventilated['AL000008'].head()

In [ ]:
x = data_pars_measurements_ventilated['AL000008']

x[x.duplicated(subset = ['PIP', 'MAP', 'PEEP', 'MV_kg', 'VTimand_kg', 'VTemand_kg',], keep = False)].head()

In [ ]:
len(x[x.duplicated(subset = ['PIP', 'MAP', 'PEEP', 'MV_kg', 'VTimand_kg', 'VTemand_kg',], keep = False)]) / len(x)

### How many % of rows have a duplicated in each recording

### 1. Sometimes the values are not updated for several consecutive measurements

In [ ]:
data_pars_measurements_ventilated['AL000268']['VTemand_resp'][:20]

In [ ]:
duplicated = {}

for case in cases:
    x = data_pars_measurements_ventilated[case]
    dupl = len(x[x.duplicated(subset = ['PIP', 'MAP', 'PEEP', 'MV_kg', 'VTimand_kg', 'VTemand_kg',], 
                              keep = 'first')]) / len(x) * 100
    duplicated[case] = dupl

duplicated = DataFrame(duplicated, index = ['pc of dupl rows']).T.sort_values(by = 'pc of dupl rows', 
                                                                              ascending = False)

In [ ]:
duplicated[duplicated['pc of dupl rows'] > 10]

In [ ]:
# Save statistics into Excel file
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'duplicated_rows_1_300.xlsx'))
duplicated.describe().to_excel(writer, 'duplicated')
writer.save()

In [ ]:
# The top 3 recordings with duplications are IPPV as in them VTemand is not measured and defaults to 0 or 'NaN'
vent_modes_ventilated.loc[duplicated[duplicated['pc of dupl rows'] > 10].index]

In [ ]:
data_pars_measurements_ventilated['AL000066'].head(10)

## Descriptive statistics on `ventilator alarms`

In [ ]:
data_pars_settings_ventilated_alarms_all = pd.concat(data_pars_alarms_ventilated, sort = True)
data_pars_settings_ventilated_alarms_all.fillna(0, inplace = True)

In [ ]:
len(data_pars_settings_ventilated_alarms_all)

In [ ]:
data_pars_settings_ventilated_alarms_all.head()

In [ ]:
data_pars_settings_ventilated_alarms_all['Alarm_susp'].value_counts()

In [ ]:
for col in data_pars_settings_ventilated_alarms_all:
    print(col, '\n', data_pars_settings_ventilated_alarms_all[col].value_counts(), '\n')

In [ ]:
data_pars_settings_ventilated_alarms_all.columns

In [ ]:
data_pars_settings_ventilated_alarms_all.sum() / len(data_pars_settings_ventilated_alarms_all) * 100

In [ ]:
data_pars_settings_ventilated_alarms_all.reset_index(level = 1, inplace = True)
grouped = data_pars_settings_ventilated_alarms_all.groupby(data_pars_settings_ventilated_alarms_all.index)

In [ ]:
alarm_counts = grouped.sum()
alarm_counts.head()

In [ ]:
alarm_pc = grouped.sum().div(grouped.size(), axis = 0) * 100
alarm_pc.head()

In [ ]:
# Save statistics into Excel file
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'alarm_stats_ventilated_1_300.xlsx'))
alarm_counts.to_excel(writer, 'alarm_counts')
alarm_pc.to_excel(writer, 'alarm_pc')
writer.save()

## Write relevant graphs about all recordings to the DATA_DUMP folder

### Graphs for ventilator recordings

#### Tidal volume of mandatory inflations

In [ ]:
%%time

par = 'VTemand_kg'
dim = 'mL/kg'; filetype = 'jpg'; dpi = 200

for case in cases:
    # print('Saving %s' % case)
    fig = plt.figure()
    fig.set_size_inches(8, 4)
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, 
                            wspace=None, hspace=0.1)
    ax = fig.add_subplot(1, 1, 1)
    data_pars_measurements_ventilated[case][par].plot(ax = ax, label = par, x_compat = True)
    ax.set_xlabel('Time', size = 14, color = 'black')
    ax.set_ylabel(dim, size = 14, color = 'black')
    # ax.set_ylim(0, 10)
    ax.set_title(case,  size = 14, color = 'black')
    ax.legend()
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
    ax.tick_params(which = 'both', labelsize=12)
            
    fig.savefig('%s/%s/%s/%s_%s.%s' % (DATA_DUMP, 'fabian_cases', case, case, par, filetype), dpi = dpi, 
        facecolor='w', edgecolor='w', orientation='portrait', 
        papertype=None, format = filetype, transparent=False, bbox_inches='tight',
        pad_inches=0.1,);
    
    plt.close()

#### Inspiratory and expiratory mandatory tidal volume

In [ ]:
%%time

pars = ['VTimand_kg', 'VTemand_kg']
name = 'VTimand_emand'
dim = 'mL/kg'; filetype = 'jpg'; dpi = 200

for case in cases:
    # print('Saving %s' % case)
    fig = plt.figure()
    fig.set_size_inches(8, 4)
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, 
                            wspace=None, hspace=0.1)
    ax = fig.add_subplot(1, 1, 1)
    data_pars_measurements_ventilated[case][pars[0]].plot(ax = ax, label = pars[0], x_compat = True)
    data_pars_measurements_ventilated[case][pars[1]].plot(ax = ax, label = pars[1], x_compat = True)
    ax.set_xlabel('Time', size = 14, color = 'black')
    ax.set_ylabel(dim, size = 14, color = 'black')
    # ax.set_ylim(0, 10)
    ax.set_title(case,  size = 14, color = 'black')
    ax.legend()
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
    ax.tick_params(which = 'both', labelsize=12)
            
    fig.savefig('%s/%s/%s/%s_%s.%s' % (DATA_DUMP, 'fabian_cases', case, case, name, filetype), dpi = dpi, 
        facecolor='w', edgecolor='w', orientation='portrait', 
        papertype=None, format = filetype, transparent=False, bbox_inches='tight',
        pad_inches=0.1,);

    plt.close()

#### Minute volume

In [ ]:
%%time

par = 'MV_kg'
dim = 'mL/kg/min'; filetype = 'jpg'; dpi = 200

for case in cases:
    # print('Saving %s' % case)
    fig = plt.figure()
    fig.set_size_inches(8, 4)
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, 
                            wspace=None, hspace=0.1)
    ax = fig.add_subplot(1, 1, 1)
    data_pars_measurements_ventilated[case][par].plot(ax = ax, label = par, x_compat = True)
    ax.set_xlabel('Time', size = 14, color = 'black')
    ax.set_ylabel(dim, size = 14, color = 'black')
    ax.set_title(case,  size = 14, color = 'black')
    ax.legend()
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
    ax.tick_params(which = 'both', labelsize=12)
            
    fig.savefig('%s/%s/%s/%s_%s.%s' % (DATA_DUMP, 'fabian_cases', case, case, par, filetype), dpi = dpi, 
        facecolor='w', edgecolor='w', orientation='portrait', 
        papertype=None, format = filetype, transparent=False, bbox_inches='tight',
        pad_inches=0.1,);

    plt.close()

#### Pressures (PIP, MAP and PEEP)

In [ ]:
%%time

pars = ['PIP', 'MAP', 'PEEP']
name = 'pressures'
dim = 'cmH2O'; filetype = 'jpg'; dpi = 200

for case in cases:
    # print('Saving %s' % case)
    fig = plt.figure()
    fig.set_size_inches(8, 4)
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, 
                            wspace=None, hspace=0.1)
    ax = fig.add_subplot(1, 1, 1)
    data_pars_measurements_ventilated[case][pars[0]].plot(x_compat = True)
    data_pars_measurements_ventilated[case][pars[1]].plot(color = 'black', linewidth = 2, x_compat = True)
    data_pars_measurements_ventilated[case][pars[2]].plot(color = 'red', x_compat = True)
    
    ax.set_xlabel('Time', size = 14, color = 'black')
    ax.set_ylabel(dim, size = 14, color = 'black')
    ax.set_title(case,  size = 14, color = 'black')
    ax.legend()
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
    ax.tick_params(which = 'both', labelsize=12)
            
    fig.savefig('%s/%s/%s/%s_%s.%s' % (DATA_DUMP, 'fabian_cases', case, case, name, filetype), dpi = dpi, 
        facecolor='w', edgecolor='w', orientation='portrait', 
        papertype=None, format = filetype, transparent=False, bbox_inches='tight',
        pad_inches=0.1,);

    plt.close()

#### Backup respiratory rate

In [ ]:
%%time

par = 'RR_set'
dim = '1/min'; filetype = 'jpg'; dpi = 200

for case in cases:
    # print('Saving %s' % case)
    fig = plt.figure()
    fig.set_size_inches(8, 4)
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, 
                            wspace=None, hspace=0.1)
    ax = fig.add_subplot(1, 1, 1)
    data_pars_settings_ventilated[case][par].plot(ax = ax, label = par, x_compat = True)
    ax.set_xlabel('Time', size = 14, color = 'black')
    ax.set_ylabel(dim, size = 14, color = 'black')
    ax.set_ylim(0, 70)
    ax.set_title(case,  size = 14, color = 'black')
    ax.legend()
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
    ax.tick_params(which = 'both', labelsize=12)
            
    fig.savefig('%s/%s/%s/%s_%s.%s' % (DATA_DUMP, 'fabian_cases', case, case, par, filetype), dpi = dpi, 
        facecolor='w', edgecolor='w', orientation='portrait', 
        papertype=None, format = filetype, transparent=False, bbox_inches='tight',
        pad_inches=0.1,);

    plt.close()

#### Actual rate and backup rate for those recordings that have SIPPV and/or SIMPSV parts

In [ ]:
%%time

pars = ['RR', 'RR_set']
name = 'rates'
dim = '1/min'; filetype = 'jpg'; dpi = 200

for case in cases:
    if vent_modes_ventilated.loc[case]['SIPPV'] > 0 or vent_modes_ventilated.loc[case]['SIMVPSV'] > 0:
        # print('Saving %s' % case)
        fig = plt.figure()
        fig.set_size_inches(8, 4)
        fig.subplots_adjust(left=None, bottom=None, right=None, top=None, 
                            wspace=None, hspace=0.1)
        ax = fig.add_subplot(1, 1, 1)
        data_pars_measurements_ventilated[case][pars[0]].plot(ax = ax, label = pars[0], x_compat = True)
        data_pars_settings_ventilated[case][pars[1]].plot(ax = ax, label = pars[1], x_compat = True)
        ax.set_xlabel('Time', size = 14, color = 'black')
        ax.set_ylabel(dim, size = 14, color = 'black')
        #ax.set_ylim(0, 70)
        ax.set_title(case,  size = 14, color = 'black')
        ax.legend()
        ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
        
        majorFmt = dates.DateFormatter('%H:%M')  
        ax.xaxis.set_major_formatter(majorFmt)
        plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        ax.tick_params(which = 'both', labelsize=12)
            
        fig.savefig('%s/%s/%s/%s_%s.%s' % (DATA_DUMP, 'fabian_cases', case, case, name, filetype), dpi = dpi, 
                facecolor='w', edgecolor='w', orientation='portrait', 
                papertype=None, format = filetype, transparent=False, bbox_inches='tight',
                pad_inches=0.1,);

        plt.close();

#### FiO2

In [ ]:
%%time

par = 'FiO2'
dim = '%'; filetype = 'jpg'; dpi = 200

for case in cases:
    # print('Saving %s' % case)
    fig = plt.figure()
    fig.set_size_inches(8, 4)
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, 
                            wspace=None, hspace=0.1)
    ax = fig.add_subplot(1, 1, 1)
    data_pars_measurements_ventilated[case][par].plot(ax = ax, label = par, x_compat = True)
    ax.set_xlabel('Time', size = 14, color = 'black')
    ax.set_ylabel(dim, size = 14, color = 'black')
    ax.set_ylim(0, 100)
    ax.set_title(case,  size = 14, color = 'black')
    ax.legend()
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
    ax.tick_params(which = 'both', labelsize=12)
            
    fig.savefig('%s/%s/%s/%s_%s.%s' % (DATA_DUMP, 'fabian_cases', case, case, par, filetype), dpi = dpi, 
        facecolor='w', edgecolor='w', orientation='portrait', 
        papertype=None, format = filetype, transparent=False, bbox_inches='tight',
        pad_inches=0.1,);

    plt.close()

#### Leak

In [ ]:
%%time

par = 'Leak'
dim = '%'; filetype = 'jpg'; dpi = 200

for case in cases:
    # print('Saving %s' % case)
    fig = plt.figure()
    fig.set_size_inches(8, 4)
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, 
                            wspace=None, hspace=0.1)
    ax = fig.add_subplot(1, 1, 1)
    data_pars_measurements_ventilated[case][par].plot(ax = ax, label = par, x_compat = True)
    ax.set_xlabel('Time', size = 14, color = 'black')
    ax.set_ylabel(dim, size = 14, color = 'black')
    ax.set_ylim(-5, 100)
    ax.set_title(case,  size = 14, color = 'black')
    ax.legend()
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
    ax.tick_params(which = 'both', labelsize=12)
            
    fig.savefig('%s/%s/%s/%s_%s.%s' % (DATA_DUMP, 'fabian_cases', case, case, par, filetype), dpi = dpi, 
        facecolor='w', edgecolor='w', orientation='portrait', 
        papertype=None, format = filetype, transparent=False, bbox_inches='tight',
        pad_inches=0.1,);

    plt.close()